In [16]:
import pandas as pd
from keras import models
from keras import layers
from keras.datasets import boston_housing
from keras.models import Model
from sklearn.model_selection import cross_val_score
from keras.layers import Input, SimpleRNN, Dense,LSTM,GRU
from keras import optimizers
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from keras import losses
from keras import metrics
from  more_itertools import unique_everseen
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import mean_squared_error
from math import sqrt
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import time as time
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import VotingRegressor
from tpot import TPOTRegressor

## Text feature extraction for single 'id' by taking hidden state of LSTM

In [ ]:
"""NOTE-> Trained on kaggle-gpu total time taken to train->1 hours approx"""
tic =time.time()
k=0
arr=[]
l=[]
tmp_0_forcast=pd.DataFrame()
for i in range(300):
    try:
        add ='tmp_'+ str(i)+'.csv'
        print(add)
        tmp=pd.read_csv('/kaggle/input/'+add)
        df_tm=tmp.set_index('id')
        count=list(tmp.groupby(['id']).size())
        id_list=list(unique_everseen(list(df_tm.index)))# to maintain sequence 
        new=dict(zip(id_list,count))
        a=0
        tmp_train=[]
        for i in id_list:
            n=new[i]
            b=a+n
            tmp_train.append(tmp[a:b])
            a=b
   
        predg=[]
        for j in range(len(tmp_train)):
            # define model
            a1=tmp_train[j].drop(['id'],axis=1)
            a = np.array(a1.values.reshape(1,a1.shape[1],a1.shape[0]))
            inputs1 = Input(shape=(a1.shape[1],a1.shape[0]))
            lstm1, state_h,state_c = LSTM(1, return_sequences=True, return_state=True)(inputs1)
            model = Model(inputs=inputs1, outputs=[lstm1, state_h,state_c])
            #print(model.predict(a))
            pred,q,e= model.predict(a,verbose=1)
            predg.append(pred)
        # define input data
        predgg=np.array(predg).reshape(len(tmp_train),300)
        predgg = pd.DataFrame(predgg)
        tmp_forcast=predgg.set_index([id_list])
        
        frames = [tmp_forcast,tmp_0_forcast]
        tmp_0_forcast=pd.concat(frames)
        print(tmp_0_forcast.shape)
    except:
        l.append(add)
        continue
toc =time.time()
print("took time in loading 241 text features by extracting hidden state of LSTM "+str((toc-tic))+" sec")
tmp_0_forcast.index.name = 'id'
tmp_0_forcast.to_csv('final_lstm.csv')
print("final_lstm shape"+str(final_lstm.shape))
"""NOTE-> Trained on kaggle-gpu total time taken to train->1 hours approx"""

## Combining text features and training data

In [2]:
final=pd.read_csv('final_lstm.csv')
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
final=final.set_index('id')
train=train.set_index('id')
test=test.set_index('id')

In [3]:
print("final text feature :"+str(final.shape))
print("training data      :"+str(train.shape))
print("test data          :"+str(test.shape))

final text feature :(5236, 300)
training data      :(4177, 362)
test data          :(1000, 361)


In [4]:
final.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
id,,,,,,,,,,,,,,,,,,,,,
M0dFSXZtQWhLUU9UTlV5abz8kaJhPpf/hj1fMYwiOnU=,-0.032031,-0.016914,0.025456,0.000597,-0.087759,-0.104635,-0.045752,-0.064834,-0.028158,-0.039532,...,-0.024026,0.036158,0.012420,0.000691,-0.020788,-0.126357,-0.069423,-0.071596,-0.046343,0.015717
MFhZNTZNRUtWbWFXZWRETz5QPVXmzHi7Qbk3H6UKr/Q=,-0.076475,-0.111015,-0.158277,-0.143028,-0.141123,-0.101019,-0.140541,-0.090161,-0.137450,-0.127279,...,-0.078248,-0.070407,-0.109594,-0.174351,-0.174958,-0.234008,-0.229291,-0.234554,-0.243427,-0.239694
MTJVTVFtTEhJTmFBWGhnOdi4QUSxr4UTXixOvQQTGuU=,0.074527,-0.001955,-0.035989,-0.080239,-0.080065,-0.007501,-0.002445,0.032705,-0.009240,0.000688,...,0.015938,-0.013455,0.022481,-0.080590,-0.052771,-0.046586,-0.019187,0.001382,-0.022918,-0.080731
NkptN2U5OHlqWENnd2xHdU2QdkNPh5dl+jDskBUMrp0=,0.001130,0.049037,-0.030378,-0.059106,-0.031642,-0.039024,-0.088188,-0.067257,-0.053612,-0.038371,...,0.041628,0.032944,0.065710,0.009169,-0.020681,-0.087809,-0.050990,-0.083981,-0.086566,-0.014950
Nm9zVm5FVDUxN3RncWZladVK0uPSGoOv6UBx3D6KIQc=,-0.059072,-0.021919,-0.014372,0.003973,-0.015170,0.029962,0.003456,0.024993,0.070024,0.056729,...,-0.037958,-0.035725,0.023259,0.003821,-0.012710,-0.009387,-0.069437,-0.074296,-0.028714,-0.117061


In [5]:
train.head()

,span,target,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,feature_07,...,feature_50_type5,feature_51_type5,feature_52_type5,feature_53_type5,feature_54_type5,feature_55_type5,feature_56_type5,feature_57_type5,feature_58_type5,feature_59_type5
id,,,,,,,,,,,,,,,,,,,,,
M00yaXQwMVJ1R1Q0RHFBVVR5BHyNyIeEd7yqxidFW7U=,17,1.025179,1.455438,0.767232,1.103207,0.355392,0.144054,1.249518,0.301203,-1.470791,...,-1.395021,-1.222626,-0.139682,0.813436,-0.340162,-0.032112,1.740059,1.000138,-1.667038,-0.128554
M01HU2pzN1dZNnVOYmhjayLgjCmIG1AyHvfRsZv+IV4=,23,0.996383,0.132304,1.724691,1.517504,0.884100,-0.897716,-0.122241,1.523423,-1.152261,...,-1.269116,0.477700,1.590249,0.166410,0.648515,0.128202,-1.853485,-0.185335,-0.933777,0.420162
M01la1Znd0ROMUU5UGJvdc5EJqBI/dj9r8C86lWkehA=,29,1.006573,0.542726,0.844240,-0.089719,0.204441,0.300044,0.564788,0.976612,0.513310,...,0.551799,1.066635,0.126409,-1.462542,1.177361,-1.142417,-0.506686,-0.741696,0.565734,-1.304175
M052cFV0RUdYT3FRMU1Kab7l27PQanylu/gYY6UjtK0=,16,0.968543,0.253863,0.799112,-1.399473,-1.979899,0.086771,-1.214792,-0.423301,1.159680,...,0.741387,-0.838532,-0.137156,0.075079,-1.395079,2.381879,-0.934174,0.775545,1.014650,1.978815
M0FFZm51b3d2a2xZOWFRWEkjj3iGrBdIeJj7omGlFz0=,4,1.006677,-0.262913,0.091079,0.799063,0.998184,-0.804963,-1.468698,-0.041668,0.533569,...,1.197475,1.162764,-0.773935,-0.962255,0.161526,-0.722540,0.490989,-1.036087,1.361248,-0.548412


In [6]:
test.head()

,span,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,feature_07,feature_08,...,feature_50_type5,feature_51_type5,feature_52_type5,feature_53_type5,feature_54_type5,feature_55_type5,feature_56_type5,feature_57_type5,feature_58_type5,feature_59_type5
id,,,,,,,,,,,,,,,,,,,,,
M0E4UFFiUkU0emNkOVlHbBAR9AAPSU6QvT49pfCItDo=,21,0.584701,0.680847,1.274691,0.822238,-0.704789,0.618635,1.008875,-1.343878,-1.315600,...,2.007172,0.793331,-0.044094,-1.012963,0.296487,-0.945259,-1.164088,-0.837606,2.070471,-0.722170
M0NLYVNZWjBjeTd1ZlBBUlngURLdbSgtBJeiqYUNosY=,15,0.294058,-0.042161,-0.634547,-1.867351,1.326732,-0.514469,-0.882709,0.358100,1.310721,...,-0.475812,-0.849995,-0.188119,0.316453,-1.096924,0.735057,0.892360,0.840361,-0.661335,0.705269
M0dDbHlMUTY5SzV0ZTBvQb9h8YABlnerdSpsViBIYVU=,19,-0.758543,1.117410,1.207818,1.524244,-0.411981,-1.722889,0.936596,-0.057886,-0.833883,...,1.953662,0.989531,-0.938730,0.034875,-0.183973,-0.538930,-0.103761,-0.057654,2.006400,-0.545562
M1JHMHhkeWlZZnJ6N01objC3E5eNJCB8DbIUbwPy/qQ=,24,0.686334,0.880205,-0.362334,-0.309460,-1.032071,1.646878,0.492463,0.236905,-1.507937,...,1.348067,0.796835,0.898317,-1.472675,0.113675,-0.596699,-2.287299,-0.851327,1.238743,-0.601774
M1dyQXB1OUZtNEpNaE5iWcovkHOs4gcR3kGXY4xfyvo=,21,0.939746,0.733183,-0.747184,-0.705871,0.760725,1.153195,-0.230777,-0.285860,1.125653,...,-1.495324,0.422166,0.814914,-0.123738,0.523765,1.085845,-1.447810,-0.022645,-2.276271,1.048187


In [7]:
s1=set(final.index)
s2=set(train.index)
s3=set(test.index)

### Checking if all the 'id' in train are present in final_lstm

In [8]:
not_in_index=list()
for i in s2:
    if i not in s1:
        not_in_index.append(i)
print("id not in final_lstm: "+str(not_in_index))

id not in final_lstm: ['SkxLeTZsZndDVVRxcHU1Vstv2wizlROZ728RsxiApzg=']


#### Removing 'id' which are not present in train

In [9]:
train=train.drop(not_in_index,axis=0)

#### Concating both the datasets

In [10]:
l_train=list(train.index)
new_final=pd.DataFrame(columns=list(final.columns))
for i in l_train:
    new_final=new_final.append(final[final.index==i])

In [11]:
#only conataing id that are present in training dataset, in same order as id's in training dataset
print(train.shape,new_final.shape)
new_final.head()

(4176, 362) (4176, 300)


,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
M00yaXQwMVJ1R1Q0RHFBVVR5BHyNyIeEd7yqxidFW7U=,-0.006561,0.001754,0.021695,-0.028540,-0.042047,-0.025066,-0.049927,-0.013242,-0.091440,-0.109428,...,0.118372,0.134226,0.169671,0.126950,0.114054,0.088835,0.038941,0.040868,0.075421,0.069241
M01HU2pzN1dZNnVOYmhjayLgjCmIG1AyHvfRsZv+IV4=,-0.056849,-0.062309,-0.085010,-0.126254,-0.152069,-0.088297,-0.016500,0.045869,-0.003839,0.024699,...,-0.038467,-0.035633,0.024682,0.076563,0.037415,0.013762,-0.027458,-0.040479,-0.003106,0.023345
M01la1Znd0ROMUU5UGJvdc5EJqBI/dj9r8C86lWkehA=,-0.026184,0.029792,0.023907,0.086234,-0.037811,0.000305,0.039828,0.024323,0.093316,0.035525,...,0.059675,0.068058,0.066053,0.016496,0.019021,-0.053866,-0.005973,-0.015370,-0.014500,-0.024072
M052cFV0RUdYT3FRMU1Kab7l27PQanylu/gYY6UjtK0=,-0.078452,-0.004653,0.050821,0.128008,0.112944,0.125822,0.106080,0.110112,0.023062,-0.032494,...,-0.029132,-0.099908,-0.043394,-0.056994,0.000821,-0.088053,-0.141273,-0.132972,-0.052653,-0.021717
M0FFZm51b3d2a2xZOWFRWEkjj3iGrBdIeJj7omGlFz0=,-0.011733,-0.002331,0.036543,0.020291,-0.034228,-0.076785,0.014478,-0.023714,0.023416,0.071067,...,-0.017916,-0.025581,-0.025134,0.014154,-0.070815,-0.116275,-0.052985,0.011968,0.014284,0.048419


In [12]:
traineco=pd.concat([train,new_final],axis=1)

In [13]:
print(traineco.shape)
traineco.head()

(4176, 662)


,span,target,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,feature_07,...,290,291,292,293,294,295,296,297,298,299
id,,,,,,,,,,,,,,,,,,,,,
M00yaXQwMVJ1R1Q0RHFBVVR5BHyNyIeEd7yqxidFW7U=,17,1.025179,1.455438,0.767232,1.103207,0.355392,0.144054,1.249518,0.301203,-1.470791,...,0.118372,0.134226,0.169671,0.126950,0.114054,0.088835,0.038941,0.040868,0.075421,0.069241
M01HU2pzN1dZNnVOYmhjayLgjCmIG1AyHvfRsZv+IV4=,23,0.996383,0.132304,1.724691,1.517504,0.884100,-0.897716,-0.122241,1.523423,-1.152261,...,-0.038467,-0.035633,0.024682,0.076563,0.037415,0.013762,-0.027458,-0.040479,-0.003106,0.023345
M01la1Znd0ROMUU5UGJvdc5EJqBI/dj9r8C86lWkehA=,29,1.006573,0.542726,0.844240,-0.089719,0.204441,0.300044,0.564788,0.976612,0.513310,...,0.059675,0.068058,0.066053,0.016496,0.019021,-0.053866,-0.005973,-0.015370,-0.014500,-0.024072
M052cFV0RUdYT3FRMU1Kab7l27PQanylu/gYY6UjtK0=,16,0.968543,0.253863,0.799112,-1.399473,-1.979899,0.086771,-1.214792,-0.423301,1.159680,...,-0.029132,-0.099908,-0.043394,-0.056994,0.000821,-0.088053,-0.141273,-0.132972,-0.052653,-0.021717
M0FFZm51b3d2a2xZOWFRWEkjj3iGrBdIeJj7omGlFz0=,4,1.006677,-0.262913,0.091079,0.799063,0.998184,-0.804963,-1.468698,-0.041668,0.533569,...,-0.017916,-0.025581,-0.025134,0.014154,-0.070815,-0.116275,-0.052985,0.011968,0.014284,0.048419


In [14]:
traineco.to_csv('traineco_lstm.csv')

### Doing same thing with the test dataset

In [16]:
not_in_index=list()
for i in s3:
    if i not in s1:
        not_in_index.append(i)
print("id not in final_lstm: "+str(not_in_index))
l_train=list(test.index)
new_final=pd.DataFrame(columns=list(final.columns))
for i in l_train:
    new_final=new_final.append(final[final.index==i])
testeco=pd.concat([test,new_final],axis=1)
testeco.to_csv('testeco_lstm.csv')
print("test data after combining :"+str(testeco.shape))
testeco.head()

id not in final_lstm: []
test data after combining :(1000, 661)


,span,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,feature_07,feature_08,...,290,291,292,293,294,295,296,297,298,299
id,,,,,,,,,,,,,,,,,,,,,
M0E4UFFiUkU0emNkOVlHbBAR9AAPSU6QvT49pfCItDo=,21,0.584701,0.680847,1.274691,0.822238,-0.704789,0.618635,1.008875,-1.343878,-1.315600,...,0.007542,-0.008553,0.030805,-0.020425,0.000551,0.023933,0.010837,0.009855,-0.026165,-0.017253
M0NLYVNZWjBjeTd1ZlBBUlngURLdbSgtBJeiqYUNosY=,15,0.294058,-0.042161,-0.634547,-1.867351,1.326732,-0.514469,-0.882709,0.358100,1.310721,...,0.080028,0.097392,0.056150,0.029915,-0.012391,-0.019810,0.034777,0.071837,0.041985,0.051523
M0dDbHlMUTY5SzV0ZTBvQb9h8YABlnerdSpsViBIYVU=,19,-0.758543,1.117410,1.207818,1.524244,-0.411981,-1.722889,0.936596,-0.057886,-0.833883,...,-0.011143,-0.035503,0.012955,0.026469,-0.034261,0.071193,0.045988,0.047851,0.037789,-0.045567
M1JHMHhkeWlZZnJ6N01objC3E5eNJCB8DbIUbwPy/qQ=,24,0.686334,0.880205,-0.362334,-0.309460,-1.032071,1.646878,0.492463,0.236905,-1.507937,...,-0.033007,-0.051173,-0.064668,-0.076753,-0.051264,-0.064924,-0.027574,-0.033489,0.025047,-0.034315
M1dyQXB1OUZtNEpNaE5iWcovkHOs4gcR3kGXY4xfyvo=,21,0.939746,0.733183,-0.747184,-0.705871,0.760725,1.153195,-0.230777,-0.285860,1.125653,...,0.040893,0.023997,0.056613,0.074068,0.081707,0.069642,0.026829,0.037812,0.063895,0.024781


## Now train the model 

In [2]:
test= pd.read_csv("testeco_lstm.csv")
train = pd.read_csv("traineco_lstm.csv")

In [3]:
gg=train.fillna(train.median())
y=gg['target']
X=gg.drop(['id','target'],axis=1)
print("X_shape:"+str(X.shape)," , y_shape :"+str(y.shape))

X_shape:(4176, 661)  , y_shape :(4176,)


In [4]:
X.head()

,span,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,feature_07,feature_08,...,290,291,292,293,294,295,296,297,298,299
0,17,1.455438,0.767232,1.103207,0.355392,0.144054,1.249518,0.301203,-1.470791,-0.934316,...,0.118372,0.134226,0.169671,0.126950,0.114054,0.088835,0.038941,0.040868,0.075421,0.069241
1,23,0.132304,1.724691,1.517504,0.884100,-0.897716,-0.122241,1.523423,-1.152261,-0.915475,...,-0.038467,-0.035633,0.024682,0.076563,0.037415,0.013762,-0.027458,-0.040479,-0.003106,0.023345
2,29,0.542726,0.844240,-0.089719,0.204441,0.300044,0.564788,0.976612,0.513310,-1.008970,...,0.059675,0.068058,0.066053,0.016496,0.019021,-0.053866,-0.005973,-0.015370,-0.014500,-0.024072
3,16,0.253863,0.799112,-1.399473,-1.979899,0.086771,-1.214792,-0.423301,1.159680,0.632020,...,-0.029132,-0.099908,-0.043394,-0.056994,0.000821,-0.088053,-0.141273,-0.132972,-0.052653,-0.021717
4,4,-0.262913,0.091079,0.799063,0.998184,-0.804963,-1.468698,-0.041668,0.533569,1.053495,...,-0.017916,-0.025581,-0.025134,0.014154,-0.070815,-0.116275,-0.052985,0.011968,0.014284,0.048419


In [9]:
X_train, X_cv, y_train, y_cv = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
from sklearn.ensemble import ExtraTreesRegressor
extra_tree = ExtraTreesRegressor(n_estimators=500,random_state=1234)
extra_tree.fit(X_train, y_train) 
ypredictions = extra_tree.predict(X_cv)
print(" Root Mean Absolute Error : ",sqrt(mean_squared_error(ypredictions, y_cv)))

 Root Mean Absolute Error :  0.012579466779304306


In [11]:
extra_tree.fit(X, y) 
test2=test.drop(['id'],axis=1)
test2=test2.fillna(test2.median())
predictions =  extra_tree.predict(test2)
pred=pd.DataFrame(predictions) 
pred=pred.set_index([test['id']])

In [12]:
pred.to_csv("submission.csv")
##submission is done after sorting in excel in decreasing order

# Note

#### Our best submission is extra_tree_500 giving accuracy-> 0.98098 on leaderboard
By Default ExtraTreesRegressor (n_estimators=500,random_state=1234)

# Other things tried for remaining submission

### By Using Hyperopt on ExtraTreesRegressor

In [17]:
def hyperopt_train_test(params):
    clf = ExtraTreesRegressor(**params)
    return cross_val_score(clf, X_train, y_train, cv=15).mean()
space4svm = {
    'max_depth':1+hp.randint('max_depth',20),
  #  'colsample_bytree':hp.uniform('colsample_bytree',0,1),
    #'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.001)),
    'min_samples_split':2+hp.randint('min_samples_split',20),
    'min_samples_leaf':1+hp.randint('min_samples_leaf',20),
    'n_jobs':1+hp.randint('n_jobs',10),
    #'min_child_weight': hp.quniform('min_child_samples', 1,20),
    #'gamma':hp.quniform('gamma',0,8)
  
    #'kernel': hp.choice('kernel', ['sigmoid', 'rbf', 'linear']),
    #'gamma': hp.uniform('gamma', 0, 5)    
}
def f(params):
    acc = hyperopt_train_test(params)
    return {'loss': -acc, 'status': STATUS_OK}
trials = Trials()
best = fmin(f, space4svm, algo=tpe.suggest, max_evals=10, trials=trials, verbose=1)
print ('best:')
print (best)
print(trials)

  0%|                                                                             | 0/10 [00:00<?, ?it/s, best loss: ?]

D:\anacondanik\envs\tf-gpu2\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)

D:\anacondanik\envs\tf-gpu2\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)

D:\anacondanik\envs\tf-gpu2\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)

D:\anacondanik\envs\tf-gpu2\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)

D:\anacondanik\envs\tf-gpu2\lib\site-packages\sklearn\ensemb

 10%|█████                                              | 1/10 [00:14<02:11, 14.60s/it, best loss: -0.1529441069457954]

D:\anacondanik\envs\tf-gpu2\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)

D:\anacondanik\envs\tf-gpu2\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)

D:\anacondanik\envs\tf-gpu2\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)

D:\anacondanik\envs\tf-gpu2\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)

D:\anacondanik\envs\tf-gpu2\lib\site-packages\sklearn\ensemb

 20%|██████████                                        | 2/10 [00:35<02:12, 16.58s/it, best loss: -0.41171232478820713]

...

 80%|████████████████████████████████████████          | 8/10 [01:51<00:28, 14.04s/it, best loss: -0.44525584264351925]

D:\anacondanik\envs\tf-gpu2\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)

D:\anacondanik\envs\tf-gpu2\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)

D:\anacondanik\envs\tf-gpu2\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)

D:\anacondanik\envs\tf-gpu2\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)

D:\anacondanik\envs\tf-gpu2\lib\site-packages\sklearn\ensemb

 90%|█████████████████████████████████████████████     | 9/10 [02:04<00:13, 13.64s/it, best loss: -0.44525584264351925]

D:\anacondanik\envs\tf-gpu2\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)

D:\anacondanik\envs\tf-gpu2\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)

D:\anacondanik\envs\tf-gpu2\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)

D:\anacondanik\envs\tf-gpu2\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)

D:\anacondanik\envs\tf-gpu2\lib\site-packages\sklearn\ensemb

100%|█████████████████████████████████████████████████| 10/10 [02:33<00:00, 18.21s/it, best loss: -0.44525584264351925]
best:
{'max_depth': 19, 'min_samples_leaf': 6, 'min_samples_split': 7, 'n_jobs': 1}


In [ ]:
extra_tree = ExtraTreesRegressor(n_estimators=500,random_state=1234,max_depth=19, min_samples_leaf=6, min_samples_split=7, n_jobs= 1)
extra_tree.fit(X_train, y_train) 
ypredictions = extra_tree.predict(X_cv)
print(" Root Mean Absolute Error : ",sqrt(mean_squared_error(ypredictions, y_cv)))

In [20]:
import lightgbm as lgb
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_cv, y_cv, reference=lgb_train)
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'num_leaves': 60,
    'learning_rate': 0.1,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 20,
    'verbose': 0
}
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=500,
                valid_sets=lgb_eval,
                early_stopping_rounds=10)
y_pred = gbm.predict(X_cv, num_iteration=gbm.best_iteration)
from sklearn.metrics import mean_squared_error
from math import sqrt
print(" Root Mean Absolute Error : ",sqrt(mean_squared_error(y_pred, y_cv)))
predictions =gbm.predict(test2)
pred_gbm=pd.DataFrame(predictions) 
pred_gbm=pred.set_index([test['id']])
pred_gbm.to_csv("submission.csv")

[1]	valid_0's l1: 0.0148526	valid_0's l2: 0.000366697
Training until validation scores don't improve for 10 rounds
[2]	valid_0's l1: 0.0146372	valid_0's l2: 0.000355597
[3]	valid_0's l1: 0.0143323	valid_0's l2: 0.000342025
[4]	valid_0's l1: 0.0141202	valid_0's l2: 0.000330206
[5]	valid_0's l1: 0.0139799	valid_0's l2: 0.00032318
[6]	valid_0's l1: 0.013897	valid_0's l2: 0.000318838
[7]	valid_0's l1: 0.0138116	valid_0's l2: 0.000314199
[8]	valid_0's l1: 0.0136327	valid_0's l2: 0.000307975
[9]	valid_0's l1: 0.0134859	valid_0's l2: 0.000301744
[10]	valid_0's l1: 0.013394	valid_0's l2: 0.000297142
[11]	valid_0's l1: 0.0133106	valid_0's l2: 0.00029328
[12]	valid_0's l1: 0.0131787	valid_0's l2: 0.000287943
[13]	valid_0's l1: 0.0130223	valid_0's l2: 0.000280686
[14]	valid_0's l1: 0.0130328	valid_0's l2: 0.00028014
[15]	valid_0's l1: 0.0129896	valid_0's l2: 0.000277789
[16]	valid_0's l1: 0.01291	valid_0's l2: 0.000274946
[17]	valid_0's l1: 0.0128587	valid_0's l2: 0.000272824
[18]	valid_0's l1: 0

In [ ]:
reg1 = GradientBoostingRegressor(random_state=123, n_estimators=200)
reg2 = RandomForestRegressor(random_state=123, n_estimators=500,max_depth=100)
reg3 = ExtraTreesRegressor(random_state=123, n_estimators=500,max_depth=100) 
ereg = VotingRegressor(estimators=[('gb',reg1) ('rf', reg2), ('et', reg3),])
ereg = ereg.fit(X_train, y_train)
y_pred = ereg.predict(X_cv)
print(" Root Mean Absolute Error : ",sqrt(mean_squared_error(y_pred, y_cv)))
predictions =  extra_tree.predict(test2)
pred_vr=pd.DataFrame(predictions) 
pred_vr=pred.set_index([test['id']])
pred_vr.to_csv("submission.csv")

In [ ]:
train1 = X[:1000]
target = y[:1000]

X_train, X_test, y_train, y_test = train_test_split(train1, target,
 train_size=0.75, test_size=0.25)

tpot = TPOTRegressor(generations=2, population_size=50, verbosity=2)
tpot.fit(X_train, y_train)
print(tpot.score(X_test, y_test))
tpot.export('tpot_boston_pipeline.py')

### Creating extra feature

In [ ]:
X['square_span']=X['span']*X['span']
X['sqrt_span']=np.sqrt(X['span'])
X['cube_span']=X['square_span']*X['span']
test['square_span']=test['span']*test['span']
test['cube_span']=test['square_span']*test['span']
test['sqrt_span']=np.sqrt(test['span'])
X_train, X_cv, y_train, y_cv = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from xgboost import XGBRegressor
my_model = XGBRegressor()
# Add silent=True to avoid printing out updates with each cycle
my_model.fit(X_train, y_train, verbose=True)
predictions = my_model.predict(X_cv)
from sklearn.metrics import mean_squared_error
from math import sqrt
print(" Root Mean Absolute Error : ",sqrt(mean_squared_error(predictions, y_cv)))
predictions = my_model.predict(test2)
pred=pd.DataFrame(predictions) 
pred=pred.set_index([test['id']])
pred_vr.to_csv("submission.csv")